In [ ]:
!pip install gluonnlp
!pip install transformers
!pip install mxnet

     |████████████████████████████████| 256kB 4.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp36-cp36m-linux_x86_64.whl size=470049 sha256=64b703cab0b2bd0c382d12c2a3078154f0adda84a219f5e8a7ff18935b52549f
  Stored in directory: /root/.cache/pip/wheels/af/60/16/1f8a40e68b85bd9bd7960e91830bca5e40cd113f3220b7e231
Successfully built gluonnlp
     |████████████████████████████████| 675kB 4.6MB/s 
     |████████████████████████████████| 1.1MB 19.8MB/s 
     |████████████████████████████████| 3.8MB 31.7MB/s 
     |████████████████████████████████| 890kB 40.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b36001864a09a993fd9da79ebd5b8a1a9b776321c8b8934835b9d328b972112e
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 68.7MB 45kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling grap

In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive') 

os.chdir('/content/gdrive/My Drive/Colab Notebooks/gpt2/KoGPT2-master')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import os
import tensorflow as tf
import gluonnlp as nlp
import pandas as pd
import re
import numpy as np

from tqdm import tqdm, tqdm_notebook
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import *
from kogpt2.pytorch_kogpt2 import get_pytorch_kogpt2_model
from kogpt2.utils import get_tokenizer
from gluonnlp.data import SentencepieceTokenizer


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2020-06-20 05:04:36--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.1, 2620:100:6016:1::a27d:101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2020-06-20 05:04:37--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfaba657b4b781af4d589ce7950.dl.dropboxusercontent.com/cd/0/get/A5_vObmCs_KgzC3mfqy1aBcEXthDGYXCEsFqTgelVMcbRLXO2iUSSLT54QfleGwpgDo4sfj1b_D551xSDagX6x_GGAhvihYFiATuYeWWp1Pe-CzR_9Y67YGmlgsKabjTnTk/file?dl=1# [following]
--2020-06-20 05:04:37--  https://ucfaba657b4b781af4d589ce7950.dl.dropboxusercontent.com/cd/0/get/A5_vObmCs_KgzC3mfqy1aBcEXthDGYXCEsFqTgelVMcbRLXO2iUSSLT54QfleGwpgDo4sfj1b_D551xSDagX6x_GGAhvihYFi

In [ ]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [ ]:
tok_path = get_tokenizer()
model_gpt, vocab, config = get_pytorch_kogpt2_model()
tok = nlp.data.BERTSPTokenizer(tok_path, vocab, lower=False)

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]
using cached model


In [ ]:
class GPT2Dataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, gpt2,
                 max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            gpt2, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
data_train = GPT2Dataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = GPT2Dataset(dataset_test, 0, 1, tok, max_len, True, False)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
for i in train_dataloader:
    print(i)
    break

[tensor([[    5, 47437, 47485,  ...,     3,     3,     3],
        [    5, 47437, 48640,  ...,     3,     3,     3],
        [    5, 47437, 47851,  ...,     3,     3,     3],
        ...,
        [    5, 47437, 47668,  ...,     3,     3,     3],
        [    5, 47437, 47703,  ...,     3,     3,     3],
        [    5, 47437, 47484,  ..., 47491, 47491,     5]], dtype=torch.int32), tensor([24, 45, 20, 37, 64, 55, 25, 64, 25, 52, 21, 50, 38, 57, 60, 18, 64, 37,
        51, 40, 45, 20, 64, 32, 20, 54, 21, 11,  7, 51, 60, 30, 19, 37, 23, 23,
        38, 40, 25, 27, 19, 64, 22, 48, 64, 20, 64, 57, 30, 64, 57, 42, 37, 57,
         8,  8,  5, 64, 10, 15, 30, 38, 33, 64], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,

In [ ]:
# attention mask 삭제
class GPT2Classifier(nn.Module):
    def __init__(self, gpt2, hidden_size = 768, num_classes = 2, dr_rate = None, params = None):
        super(GPT2Classifier, self).__init__()
        self.gpt2 = gpt2 # pre_trained model
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def forward(self, token_ids, valid_length, segment_ids):

        _, hidden_output = self.gpt2(input_ids=token_ids, token_type_ids=segment_ids.long())
        pooled_output = hidden_output[:,-1]
        # pooler_output = pooler_out[0][:,-1] ## transformers 모듈을 사용할 경우

        if self.dr_rate:
            out = self.dropout(pooled_output)

        return self.classifier(out)

In [ ]:
model = GPT2Classifier(model_gpt, dr_rate = 0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    with open('checkpoint/checkpoint_{}.model'.format(str(e + 1).zfill(2)), 'wb') as f:
        torch.save(model.state_dict(), f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


epoch 1 batch id 1 loss 0.7034438252449036 train acc 0.5
epoch 1 batch id 201 loss 0.670179545879364 train acc 0.5051305970149254
epoch 1 batch id 401 loss 0.6943207383155823 train acc 0.5105595386533666
epoch 1 batch id 601 loss 0.6621103882789612 train acc 0.5281301996672213
epoch 1 batch id 801 loss 0.639626145362854 train acc 0.5628706304619226
epoch 1 batch id 1001 loss 0.4941474497318268 train acc 0.5921734515484516
epoch 1 batch id 1201 loss 0.5970379710197449 train acc 0.6184950041631974
epoch 1 batch id 1401 loss 0.4328145384788513 train acc 0.6415060670949322
epoch 1 batch id 1601 loss 0.5228791236877441 train acc 0.6615884603372892
epoch 1 batch id 1801 loss 0.31627383828163147 train acc 0.6787635341476957
epoch 1 batch id 2001 loss 0.4400116205215454 train acc 0.694059220389805
epoch 1 batch id 2201 loss 0.30356714129447937 train acc 0.7068661971830986

epoch 1 train acc 0.715277037116041


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.852861253196931


epoch 2 batch id 1 loss 0.4810100495815277 train acc 0.78125
epoch 2 batch id 201 loss 0.26773202419281006 train acc 0.8475590796019901
epoch 2 batch id 401 loss 0.3781464397907257 train acc 0.8482699501246883
epoch 2 batch id 601 loss 0.4141322672367096 train acc 0.8505615640599001
epoch 2 batch id 801 loss 0.3310777246952057 train acc 0.8518453495630461
epoch 2 batch id 1001 loss 0.33977457880973816 train acc 0.8530063686313686
epoch 2 batch id 1201 loss 0.2807278037071228 train acc 0.8543661532056619
epoch 2 batch id 1401 loss 0.3062792122364044 train acc 0.8550811920057102
epoch 2 batch id 1601 loss 0.3502008616924286 train acc 0.8568082448469706
epoch 2 batch id 1801 loss 0.24568265676498413 train acc 0.8581603970016657
epoch 2 batch id 2001 loss 0.30714356899261475 train acc 0.8601402423788106
epoch 2 batch id 2201 loss 0.2332441210746765 train acc 0.8612846433439346

epoch 2 train acc 0.8624057878270762


...


epoch 3 test acc 0.8795756074168798


epoch 4 batch id 1 loss 0.44419124722480774 train acc 0.828125
epoch 4 batch id 201 loss 0.17259781062602997 train acc 0.8962220149253731
epoch 4 batch id 401 loss 0.29218193888664246 train acc 0.8956125311720698
epoch 4 batch id 601 loss 0.33191007375717163 train acc 0.8969945923460898
epoch 4 batch id 801 loss 0.28981003165245056 train acc 0.897042759051186
epoch 4 batch id 1001 loss 0.26277148723602295 train acc 0.896416083916084
epoch 4 batch id 1201 loss 0.20423045754432678 train acc 0.8969348459616986
epoch 4 batch id 1401 loss 0.2309565544128418 train acc 0.8969151498929336
epoch 4 batch id 1601 loss 0.3503631353378296 train acc 0.8973590724547158
epoch 4 batch id 1801 loss 0.16731536388397217 train acc 0.8978432121043864
epoch 4 batch id 2001 loss 0.24482685327529907 train acc 0.8985741504247876
epoch 4 batch id 2201 loss 0.24589766561985016 train acc 0.8989095865515675

epoch 4 train acc 0.8995751564277588



epoch 4 test acc 0.8807544757033248


epoch 5 batch id 1 loss 0.38590848445892334 train acc 0.890625
epoch 5 batch id 201 loss 0.2155696451663971 train acc 0.908193407960199
epoch 5 batch id 401 loss 0.23470740020275116 train acc 0.9071851620947631
epoch 5 batch id 601 loss 0.31197819113731384 train acc 0.9076539101497504
epoch 5 batch id 801 loss 0.2587742209434509 train acc 0.9074204119850188
epoch 5 batch id 1001 loss 0.26625826954841614 train acc 0.907249000999001
epoch 5 batch id 1201 loss 0.15693943202495575 train acc 0.907173709408826
epoch 5 batch id 1401 loss 0.22300291061401367 train acc 0.9070529978586723
epoch 5 batch id 1601 loss 0.23938360810279846 train acc 0.907508978763273
epoch 5 batch id 1801 loss 0.2225169837474823 train acc 0.907638117712382
epoch 5 batch id 2001 loss 0.20323260128498077 train acc 0.9083583208395802
epoch 5 batch id 2201 loss 0.15271222591400146 train acc 0.9088482507950931

epoch 5 train acc 0.9093718892207054



epoch 5 test acc 0.8818134590792839
